## CNN on CIFAR
#### By MMA

In [14]:
import tensorflow as tf
import numpy as np

#### Training Parameter

In [15]:
LEARNING_RATE = 0.001
BATCH_SIZE = 128
EPOCHS = 1000

#### Read Data

In [4]:
import input_data
cifar = input_data.read_data_sets("cifar-10/", one_hot=True)
cifar.normalize_data()
d = cifar.data
print(d[:10][0].shape)

(3072,)


#### Network Parameter

In [18]:
num_input = 3072
num_classes = 10

#### Tensorflow Graph Input

In [19]:
X = tf.placeholder(dtype=tf.float32, shape=(None, num_input), name="Inputs")
Y = tf.placeholder(dtype=tf.float32, shape=(None, num_classes), name="Lables")
keep_prob = tf.placeholder(dtype=tf.float32) # DropOut ( Keep Probability )

#### Create some wrappers

In [20]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [21]:
def maxpool2d(x, k=2, stride=1):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, stride, stride, 1], padding='SAME')

#### Create Model

In [22]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 32, 32, 3])
    
    # Convolution Layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling
    conv1 = maxpool2d(conv1, k=3, stride=1)
    
    # Convolution Layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling
    conv2 = maxpool2d(conv2, k=3, stride=1)
    
    # Fully Connected Layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Output
    constant = np.full((1, num_classes), 0.001)
    output = tf.add(tf.matmul(fc1, weights['output']), biases['output']) + constant
    return output

#### Store Layer Weights and Biases

In [23]:
weights = {
    'wc1': tf.Variable(tf.truncated_normal([5, 5, 3, 64], 0, 0.01)),
    'wc2': tf.Variable(tf.truncated_normal([5, 5, 64, 64], 0, 0.01)),
    'wd1': tf.Variable(tf.truncated_normal([32*32*64, 512], 0, 0.01)),
    'output': tf.Variable(tf.truncated_normal([512, num_classes], 0, 0.01))
}

biases = {
    'bc1': tf.Variable(tf.zeros([64])),
    'bc2': tf.Variable(tf.zeros([64])),
    'bd1': tf.Variable(tf.zeros([512])),
    'output': tf.Variable(tf.zeros([num_classes]))
}

#### Construct Model

In [24]:
logits = conv_net(X, weights, biases, keep_prob)
constant = np.full((1, num_classes), 0.001)
prediction = tf.nn.softmax(logits) + constant

#### Define loss and optimizer

In [25]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32), name="accuracy")

#### Start Training

In [26]:
init = tf.global_variables_initializer()
sess_cnn = tf.Session()
sess_cnn.run(init)
for i in range(EPOCHS):
    batch_xs, batch_ys = cifar.train.next_batch(BATCH_SIZE)
    sess_cnn.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.8})
    if i % 50 == 0:
        print(i)
        acc = sess_cnn.run(accuracy_op, feed_dict={X: batch_xs, Y: batch_ys, keep_prob:1.0})
        print(acc)
        # b1, b2 = (sess_cnn.run([loss_train_summary_op, acc_train_summary_op],
        #                        feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 1.0}))
        # filewriter.add_summary(b1, i)
        # filewriter.add_summary(b2, i)

        # batch_xs_v, batch_ys_v = cifar.validation.next_batch(BATCH_SIZE)
        # b1, b2 = (sess_cnn.run([acc_val_summary_op, loss_val_summary_op],
        #                        feed_dict={X: batch_xs_v, Y: batch_ys_v, keep_prob: 1.0}))
        # filewriter.add_summary(b1, i)
        # filewriter.add_summary(b2, i)

0


0.15625


50


0.2109375


100


...

750
nan


800
nan


850
nan


900
nan


950
nan


In [28]:
[accuracy] = sess_cnn.run([accuracy_op], feed_dict={X: cifar.test.images[:256],
                                                    Y: cifar.test.labels[:256],
                                                    keep_prob: 1.0})
print("The accuracy of test:" + str(accuracy))

The accuracy of test:0.078125
